Neurons are the simplest units of our nervous system. Complex communication between neurons is what gives rise to all brain functions. Scientists in the 19th century believed that neurons communicate using electricity, but it was not until 1974 that Louis Edouard Lapique came up with a simple mathematical model of how neurons send electric signals to each other. This model is called the Leaky Integrate-and-Fire Neuron (LIF). The components of the model are:

1. Membrane Potential (V) which, when multiplied by the Capacitance (C), gives the current electric charge of the neuron. Capacitance is how much charge a neuron can store (in the form of charged ions), and potential is the energy that can push the charges in the neuron for ions (electric signal) to flow.

2. Resting Potential ($E_L$) which is the membrane potential at rest. When this potential goes above a certain threshold ($V_t$) by the action of external input current (I), the neuron fires, and a
spike is generated.

3. External Input (I) which describes the current that feeds into the neuron, part of which is used to increase the neurons’ electrical potential and thus ion charge, but some of it gets leaked outside the neuron. The leakage is determined by Leak Conductance ($g_L$) which is the tendency of the neuron to leak charge when it is too charged, i.e. V >> $E_L$.

4. Diving the capacitance by the leak conductance values gives rise to a property called the Time Constant ($𝛕_m$), which is the time it takes for a neuron to charge and fire a spike, controlled by its ability to store charge versus its tendency to leak it off which delays the charging process and thus the increase of V until it passes V_t to fire a spike.

Now that we presented the model components, let’s discuss the equations that combine them together to explain the model.

$C_m dV/dt = -g_L(V-E_L)  + I$

$𝜏_m dV/dt = -(V-E_L)  + I / g_L$

The first equation expresses the change in neuron charge over time. This is directly proportional to the external current and decreases as the leak conductance increases because more charge is lost. The leak increases the more the neuron is charged far from its resting state. The second equation is a different version of the first by dividing it over $g_L$. It simply shows that the neuron gets charged over time by the external current with a leak, and the change in charge is inversely proportional to the time constant of the neuron. Therefore, the bigger 𝜏m gets, the longer it takes to generate a spike. The interactive figure below shows this effect.

**Imports**

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import logging
import ipywidgets as widgets

**Figure helpers**

In [14]:
logging.getLogger('matplotlib.font_manager').disabled = True
%config InlineBackend.figure_format = 'retina'
# use NMA plot style
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")
mod_layout = widgets.Layout()

def default_pars(**kwargs):
  pars = {}

  # typical neuron parameters#
  pars['V_th'] = -55.     # spike threshold [mV]
  pars['V_reset'] = -75.  # reset potential [mV]
  pars['tau_m'] = 10.     # membrane time constant [ms]
  pars['g_L'] = 10.       # leak conductance [nS]
  pars['V_init'] = -75.   # initial potential [mV]
  pars['E_L'] = -75.      # leak reversal potential [mV]
  pars['ref_t'] = 2.       # refractory time (ms)

  # simulation parameters #
  pars['T'] = 400.  # Total duration of simulation [ms]
  pars['dt'] = .1   # Simulation time step [ms]

  # external parameters if any #
  for k in kwargs:
    pars[k] = kwargs[k]

  pars['range_t'] = np.arange(0, pars['T'], pars['dt'])  # Vector of discretized time points [ms]

  return pars


parameters = default_pars()


def plot_volt_trace(pars, v, sp):
  """
  Plot trajetory of membrane potential for a single neuron

  Expects:
  pars   : parameter dictionary
  v      : volt trajetory
  sp     : spike train

  Returns:
  figure of the membrane potential trajetory for a single neuron
  """

  V_th = pars['V_th']
  dt, range_t = pars['dt'], pars['range_t']
  if sp.size:
    sp_num = (sp / dt).astype(int) - 1
    v[sp_num] += 20  # draw nicer spikes

  plt.plot(pars['range_t'], v, 'b')
  plt.axhline(V_th, 0, 1, color='k', ls='--')
  plt.xlabel('Time (ms)')
  plt.ylabel('V (mV)')
  plt.legend(['Membrane\npotential', r'Threshold V$_{\mathrm{th}}$'],
             loc=[1.05, 0.75])
  plt.ylim([-80, -40])
  plt.show()

**Model Helpers**

In [15]:
def run_LIF(pars, Iinj, stop=False):
  """
  Simulate the LIF dynamics with external input current

  Args:
    pars       : parameter dictionary
    Iinj       : input current [pA]. The injected current here can be a value
                 or an array
    stop       : boolean. If True, use a current pulse

  Returns:
    rec_v      : membrane potential
    rec_sp     : spike times
  """

  # Set parameters
  V_th, V_reset = pars['V_th'], pars['V_reset']
  tau_m, g_L = pars['tau_m'], pars['g_L']
  V_init, E_L = pars['V_init'], pars['E_L']
  dt, range_t = pars['dt'], pars['range_t']
  Lt = range_t.size
  ref_t = pars['ref_t']

  # Initialize voltage
  v = np.zeros(Lt)
  v[0] = V_init

  # Set current time course
  Iinj = Iinj * np.ones(Lt)

  # If current pulse, set beginning and end to 0
  if stop:
    Iinj[:int(len(Iinj) / 2) - 1000] = 0
    Iinj[int(len(Iinj) / 2) + 1000:] = 0

  # Loop over time
  rec_spikes = []  # record spike times
  tr = 0.  # the count for refractory duration

  for it in range(Lt - 1):

    if tr > 0:  # check if in refractory period
      v[it] = V_reset  # set voltage to reset
      tr = tr - 1 # reduce running counter of refractory period

    elif v[it] >= V_th:  # if voltage over threshold
      rec_spikes.append(it)  # record spike event
      v[it] = V_reset  # reset voltage
      tr = ref_t / dt  # set refractory time

    # Calculate the increment of the membrane potential
    dv = (-(v[it] - E_L) + Iinj[it] / g_L) * (dt / tau_m)

    # Update the membrane potential
    v[it + 1] = v[it] + dv

  # Get spike times in ms
  rec_spikes = np.array(rec_spikes) * dt

  return v, rec_spikes

**Interactive Widget**

In [16]:
# Reminder: Ensure this cell is executed to activate the interactive widget!

# Set the layout width
mod_layout.width = '450px'

# Create interactive widgets
@widgets.interact(
    I_dc=widgets.FloatSlider(value=50., min=0., max=300., step=10., layout=mod_layout),
    tau_m=widgets.FloatSlider(value=10., min=2., max=20., step=2., layout=mod_layout)
)

def visualize_LIF_response(I_dc=200., tau_m=10.):
    """
    Visualize the response of an LIF neuron for given parameters.

    Args:
    - I_dc : Direct current input.
    - tau_m: Membrane time constant.
    """
    # Get default parameters and update tau_m
    parameters = default_pars(T=100.)
    parameters['tau_m'] = tau_m

    # Simulate the LIF neuron
    voltage, spikes = run_LIF(parameters, Iinj=I_dc)

    # Plot the voltage trace
    plot_volt_trace(parameters, voltage, spikes)
    plt.show()


interactive(children=(FloatSlider(value=50.0, description='I_dc', layout=Layout(width='450px'), max=300.0, ste…

We can see that increasing tau_m, the membrane time constant, makes us need more I_dc for enough effective charge to accumulate to cause an action potential spike.

Now let's add one more layer of real-world complexity into the model. In-vivo brain electricity is not as uniform as DC current is. It is noisy and includes fluctuations. The consider this we add Gaussian noise to electric current in our model. So, the interactive figure below explores the effect of adding Gaussian noise to the input current of neurons and the extent to which the variance of that noisy current influences spiking.

In [17]:
def my_GWN(pars, mu, sig, myseed=False):
  """
  Function that generates Gaussian white noise input

  Args:
    pars       : parameter dictionary
    mu         : noise baseline (mean)
    sig        : noise amplitute (standard deviation)
    myseed     : random seed. int or boolean
                 the same seed will give the same
                 random number sequence

  Returns:
    I          : Gaussian white noise input
  """

  # Retrieve simulation parameters
  dt, range_t = pars['dt'], pars['range_t']
  Lt = range_t.size

  # Set random seed
  if myseed:
      np.random.seed(seed=myseed)
  else:
      np.random.seed()

  # Generate GWN
  # we divide here by 1000 to convert units to sec.
  I_gwn = mu + sig * np.random.randn(Lt) / np.sqrt(dt / 1000.)

  return I_gwn

help(my_GWN)

Help on function my_GWN in module __main__:

my_GWN(pars, mu, sig, myseed=False)
    Function that generates Gaussian white noise input
    
    Args:
      pars       : parameter dictionary
      mu         : noise baseline (mean)
      sig        : noise amplitute (standard deviation)
      myseed     : random seed. int or boolean
                   the same seed will give the same
                   random number sequence
    
    Returns:
      I          : Gaussian white noise input



In [18]:
mod_layout.width = '450px'
@widgets.interact(
    mu_gwn=widgets.FloatSlider(200., min=100., max=300., step=5.,
                               layout=mod_layout),
    sig_gwn=widgets.FloatSlider(2.5, min=0., max=5., step=.5,
                                layout=mod_layout)
)


def diff_GWN_to_LIF(mu_gwn, sig_gwn):
  pars = default_pars(T=100.)
  I_GWN = my_GWN(pars, mu=mu_gwn, sig=sig_gwn)
  v, sp = run_LIF(pars, Iinj=I_GWN)
  plt.figure(figsize=(12, 4))
  plt.subplot(121)
  plt.plot(pars['range_t'][::3], I_GWN[::3], 'b')
  plt.xlabel('Time (ms)')
  plt.ylabel(r'$I_{GWN}$ (pA)')
  plt.subplot(122)
  plot_volt_trace(pars, v, sp)
  plt.tight_layout()
  plt.show()

interactive(children=(FloatSlider(value=200.0, description='mu_gwn', layout=Layout(width='450px'), max=300.0, …

Here we can see that increasing the noise level in the input current (standard deviation of the Gaussian noise) helps neurons with lower average DC component of the input current to reach their threshold potential and fire compared to small fluctuations that don't help push the voltage to their firing threshold. It is important to note that this is an important property in real-world neurons which should explain any discrepancy between simulated spiking rates from just DC current and the actual neural behavior. We can see this more clearly in the interactive plot below.

In [19]:
mod_layout.width = '450px'
@widgets.interact(
    sig_gwn=widgets.FloatSlider(3.0, min=0., max=6., step=0.5,
                                layout=mod_layout)
)


def diff_std_affect_fI(sig_gwn):
  pars = default_pars(T=1000.)
  I_mean = np.arange(100., 400., 10.)
  spk_count = np.zeros(len(I_mean))
  spk_count_dc = np.zeros(len(I_mean))

  for idx in range(len(I_mean)):
      I_GWN = my_GWN(pars, mu=I_mean[idx], sig=sig_gwn, myseed=2020)
      v, rec_spikes = run_LIF(pars, Iinj=I_GWN)
      v_dc, rec_sp_dc = run_LIF(pars, Iinj=I_mean[idx])
      spk_count[idx] = len(rec_spikes)
      spk_count_dc[idx] = len(rec_sp_dc)

  # Plot the F-I curve i.e. Output firing rate as a function of input mean.
  plt.figure()
  plt.plot(I_mean, spk_count, 'k',
           label=r'$\sigma_{\mathrm{GWN}}=%.2f$' % sig_gwn)
  plt.plot(I_mean, spk_count_dc, 'k--', alpha=0.5, lw=4, dashes=(2, 2),
           label='DC input')
  plt.ylabel('Spike count')
  plt.xlabel('Average injected current (pA)')
  plt.legend(loc='best')
  plt.show()

interactive(children=(FloatSlider(value=3.0, description='sig_gwn', layout=Layout(width='450px'), max=6.0, ste…

We can see that when we set sigma to zero, the spike count follows a pattern similar to that with a DC input current, where the threshold of the injected current that causes a spike is high. However, as we increase signal noise, smaller currents become able to induce spikes.

voila